In [1]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
from data_agent import *
import numpy as np
from data_util import *
import tensorflow as tf
from sklearn_model_hyper import *
import pandas as pd
from stock_agent import *
from backtest import *
from joblib import dump, load
from datetime import datetime, timedelta
from tec_an import *
from bitstamp import *
from process_live_trade import *
from model_winner_select import *
from eval_model import *
from cache_providers import *

base_dir = "data/"

In [2]:
currency = "btcusd"

full_data = base_dir + "backtest/"
full_data = base_dir + "4jul21/"
#full_data = base_dir + "btcusd/"
data_gen = DataGenerator(random = False, base_dir = full_data)
data_gen.rewind()
data_count = (data_gen.steps - 100)
#data_count = 200000
raw = data_gen.next()

minute = 30

hot_load = True

In [3]:
path = "model/"
files = os.listdir(path)
models = []
for file in files:
    models.append(f"{path}{file}")

models = ['model/rand']
#models = ['model/quadra']
    
val_start = int(raw['timestamp'])

In [4]:
winner = get_best_model(
    minutes_list=[minute],
    result_paths=models,
    currency_list=[currency],
    timestamp = val_start,
    winner_path = None
)

model/rand - 200
Pre selected: 109


100%|██████████| 109/109 [02:08<00:00,  1.18s/it]

Selected: 13
Pipeline(steps=[('normalizer', Normalizer()),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=False, criterion='entropy',
                                        max_features=0.1, random_state=42))])
btcusd -> 100.33378527523628 | 2 - 0
{'recall': 0.42833052276559863, 'precision': 0.436426116838488, 'f1': 0.4323404255319149, 'accuracy': 0.5374479889042996}

Pipeline(steps=[('normalizer', Normalizer()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFE(estimator=ExtraTreesClassifier(max_features=0.3,
                                                                     random_state=42),
                                      step=0.7500000000000001)),
                                 ('m',
                                  RandomForestClassifier(bootstrap=False,
                                                         criterion='entropy',
                                      

In [5]:
result = winner
if (winner == None):
    result = results[-1]
model = result['model']
windows = result['window']
minutes = result['minutes']
step = result['step']
profit = result['profit']
print(f"Minutes={minutes} Window={windows} Step={step} | {profit}")
print(f"{model}")


Minutes=30 Window=40 Step=1 | 103.5047554015294
RandomForestClassifier(bootstrap=False, criterion='entropy', max_features=0.1,
                       random_state=42)


In [6]:
import numpy as np
import matplotlib.pyplot as plt
def get_charts(stock_data):
    fig = plt.figure()
    ax = plt.axes()
    x = stock_data
    plt.plot(x, linewidth=1)
    #plt.grid(b=True, color='aqua', alpha=0.3, linestyle='-.', linewidth=2)
    plt.show()


In [7]:

on_raw_data = lambda raw: print(raw)

data_gen.rewind()

first = data_gen.next()

In [9]:
agent_reference, back_reference, stock_reference = get_agent(minutes = minutes,
                                win = windows,
                                step = step,
                                currency = currency,
                                hot_load = hot_load,
                                model = model,
                                timestamp = int(first['timestamp']),
                                simulate_on_price = True,
                                verbose = False)
prices = []
agent_reference.on_closed_price = lambda price: prices.append(price)
#agent.ohlc['open'].plot()

In [10]:
data_gen.rewind()


reference_list = []
#data_count
for i in tqdm(range(data_count)):
    raw = data_gen.next()
    price = raw[PRICE_KEY]
    amount = raw[AMOUNT_KEY]
    timestamp = raw[TIMESTAMP_KEY]
    timestamp_pd = pd.to_datetime(timestamp, unit='s')
    reference_list.append([timestamp_pd, price])


  0%|          | 0/301794 [00:00<?, ?it/s]

In [11]:
DATE = 'Date'
df = pd.DataFrame(agent_reference.list, columns = [DATE, CLOSE])
df = df.set_index(pd.DatetimeIndex(df[DATE]))

In [16]:
time = df[CLOSE].resample(agent_reference.resample)
ohlc = time.ohlc()
print("All data reference")
#ohlc.plot()
#ohlc

All data reference


In [17]:
cache = CacheProvider([currency],  verbose = True)


In [14]:

online = cache.get_provider(minutes, windows, int(first['timestamp']), int(raw['timestamp']))

eval_model(
    model = model, 
    currency = currency, 
    step = step,
    verbose = False, 
    provider = online,
    hot_load_total = 100,
    cache = CacheProvider(currency_list = [currency], verbose=False)
    )

2021-08-21 00:17:33.345335 CacheProvider: Added: 30-40


(<stock_agent.BackTest at 0x7fbaa25b7700>,
 {'recall': 1.0,
  'precision': 0.23529411764705882,
  'f1': 0.38095238095238093,
  'accuracy': 0.4090909090909091,
  'roc_auc': 0.6388888888888888})

In [15]:
reference_list[0]

[Timestamp('2021-07-04 22:49:41'), 35330.0]

In [18]:
agent_raw, back_raw, stock_raw = get_agent(minutes = minutes,
                                win = windows,
                                step = step,
                                currency = currency,
                                hot_load = hot_load,
                                model = model,
                                verbose = True,
                                timestamp = int(first['timestamp']),
                                simulate_on_price = True)
prices = []

###### Past report(201): 2021-06-30 18:00:00(1625076000) - 2021-07-04 22:00:00(1625436000) ######
2.24684% -> 102.24683675801671
Positive: 31 Negative: 8
###### - ######


In [19]:
data_gen.rewind()
agent_raw.save_history = True
stock_raw.verbose = True
#data_count
for i in range(data_count):
    raw = data_gen.next()
    agent_raw.on_new_raw_data(raw)

2021-08-21 00:18:31.215829 ModelAgent(35330.0): 2021-07-04 22:49:41(1625438981) UP
2021-08-21 00:18:31.215977 BackTest: Bought: 35330.0
2021-08-21 00:18:32.583269 ModelAgent(35340.23): 2021-07-04 23:00:04(1625439604) DOWN
2021-08-21 00:18:32.583482 BackTest: SOLD >>>> Result: PROFIT 0.029 total: 100.02895556184546
2021-08-21 00:18:36.864331 ModelAgent(35415.34): 2021-07-04 23:30:08(1625441408) DOWN
2021-08-21 00:18:39.540914 ModelAgent(35247.7): 2021-07-05 00:00:17(1625443217) DOWN
2021-08-21 00:18:42.286101 ModelAgent(34922.82): 2021-07-05 00:30:08(1625445008) UP
2021-08-21 00:18:42.286228 BackTest: Bought: 34922.82
2021-08-21 00:18:44.308242 ModelAgent(34898.82): 2021-07-05 01:00:40(1625446840) DOWN
2021-08-21 00:18:44.308456 BackTest: SOLD >>>> Result: LOSS -0.0687 total: 99.96023258930202
2021-08-21 00:18:46.572951 ModelAgent(34639.91): 2021-07-05 01:30:15(1625448615) DOWN
2021-08-21 00:18:48.784485 ModelAgent(34560.8): 2021-07-05 02:00:08(1625450408) DOWN
2021-08-21 00:18:50.97451

In [ ]:
val_start = int(first['timestamp'])
val_end = int(raw['timestamp'])

start_string = pd.to_datetime(val_start, unit='s')
end_string = pd.to_datetime(val_end, unit='s')

print(f"{start_string}({val_start}) -> {end_string}({val_end})")

In [ ]:
back_raw.on_down(back_raw.buy_price, back_raw.buy_price)
back_raw.report()

In [ ]:
agent_raw.ohlc[-50:]["open"].plot()
#agent.ohlc[1:]

In [ ]:
online = load_val_data(minutes = 5, 
                 window = window,
                 val_start = val_start,
                 val_end = val_end,
                  currency_list = [currency]
                      )


In [ ]:
online.load_val_cache([currency],1629215891,  1629229205)

In [ ]:
x_list, price_list, time_list = online.load_val_data(currency)

In [ ]:
get_charts(price_list)
print(len(price_list))

In [ ]:
agent_array, back_array, stock_array = get_agent(minutes = minutes,
                                win = window,
                                step = step,
                                currency = currency,
                                hot_load = hot_load,
                                model = model,
                                timestamp = int(first['timestamp']),
                                simulate_on_price = True)

In [ ]:
agent_array.save_history = True
for idx in range(len(price_list)):
    price = price_list[idx]
    time = time_list[idx]
    order = [[f"{price}", f"{price}"]]
    amount = 0.0
    agent_array.process_data(price, amount, time, order, order)

In [ ]:
back_array.on_down(back_array.buy_price, back_array.buy_price)
back_array.report()

In [ ]:
def eval_step_(model, train_set, step, provider, verbose = False):

    valX, valY, times = provider.load_val_data(train_set)
    
    x, y, closed_prices = get_sequencial_data(valX, valY, step)
    
    history = []
    
    preds = model.predict(np.array(x))
    
    #for idx in range(len(x)):
    #    print(f"{times[idx]} -> {closed_prices[idx]} {y[idx]} ")

    metrics = {}
    metrics["recall"] = recall_score(y, preds)
    metrics["precision"] = precision_score(y, preds)
    metrics["f1"] = f1_score(y, preds)
    metrics["accuracy"] = accuracy_score(y, preds)
    metrics["roc_auc"] = roc_auc_score(y, preds)
    
    back = BackTest(value = 100, 
                    verbose = True, 
                    sell_on_profit = True,
                    pending_sell_steps = step)
    
    back = backtest_model(model, valX, valY, back)
    
    for idx in range(len(times)):
        timestamp = times[idx]
        price = closed_prices[idx]
        y = model.predict([valX[idx]])
        is_up = False
        if (y[0] > 0.5):
            is_up = True
        history.append((timestamp, price, x, is_up))
    
    get_charts(valY)

    
    return back, metrics, history
    

In [ ]:
back_eval, metr, history = eval_step_(model, currency, step, online, verbose = False)
back_eval.on_down(back_eval.buy_price, back_eval.buy_price)
back_eval.report()

In [ ]:
for data in history:
    print(f"{data[0]} - {data[1]} - {data[3]}")

In [ ]:
print(metrics)


In [ ]:
back_raw.report()
#-1.78465% -> 98.21535030676218

In [ ]:
back_array.report()
#-1.89717% -> 98.10282664807247

In [ ]:
agent_raw.report()

In [ ]:
agent_array.report()